#Introduction

Business intelligence is very important nowadays. It helps you to know better of your company including how the sales recently, which item is going to out of stock and most importantly, what is the profit.

In this project, I am going to demonstrate how to use data from a pizza shop along with SQL to create 3 dashboards for business intelligence.

##Data content

There are 10 tables in the dataset.

Table name | Content | Columns
--- | --- | ---
Address | Customers' address and zip code | 'add_id', 'delivery_address1', 'delivery_address2', 'delivery_city', 'delivery_zipcode'
Customers | First and last name | 'cust_id', 'cust_firstname', 'cust_lastname'
Ingredient | Ingredient name, weight, unit and price | 'ing_id', 'ing_name', 'ing_weight', 'ing_meas', 'ing_price'
Inventory | Quantity of inventory | 'inv_id', 'item_id', 'quantity'
Item | Item name, code, category, size and price | 'item_id', 'sku', 'item_name', 'item_cat', 'item_size', 'item_price'
Orders | Order create time, item ordered, quantity, customer ID, delivery/pick up, address ID | 'row_id', 'order_id', 'created_at', 'item_id', 'quantity', 'cust_id', 'delivery', 'add_id'
Recipe | Recipe ID, ingredient ID, required quantity | 'row_id', 'recipe_id', 'ing_id', 'quantity'
Rota | Duty date, shift ID, staff ID | 'row_id', 'rota_id', 'date', 'shift_id', 'staff_id'
Shift | Day of week, start time and end time | 'shift_id', 'day_of_week', 'start_time', 'end_time'
Staff | Staff first and last name, position, hourly rate | 'staff_id', 'first_name', 'last_name', 'position', 'hourly_rate'

##Dashboard 1

This dashboard contains the basic information about the pizza shop. It tells whether the pizza shop is earning or losing money.
1.   Total number of order
2.   Total sales
3.   Total item sold
4.   Average order value
5.   Sales by category
6.   Top selling item
7.   Orders by hour
8.   Sales by hour
9.   Orders by address
10.  Orders by delivery/pick up

##Dashboard 2

The second dashboard is about the material cost and stock information to keep the pizza shop running.
1.   Total quantity required for orders by ingredient
2.   Total cost of ingredients for orders
3.   Calculated cost of pizza
4.   Percentage stock remaining by ingredient
5.   List of ingredients to re-order based on remaining inventory

##Dashboard 3

The last dashboard is going to capture the human resource cost of the pizza shop.
1.   Hours worked by staff member
2.   Total hours worked
3.   Cost per staff member
4.   Total staff cost

##Import data

In [95]:
# connect google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [96]:
# import library
import sqlite3
import pandas as pd
import numpy as np

In [97]:
# define functions

def pd_to_sqlDB(input_df: pd.DataFrame, table_name: str, db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table
    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 2: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()

    # Step 3: Create Table
    sql_string = f"""CREATE TABLE IF NOT EXISTS {table_name} ({cols_string});"""
    cur.execute(sql_string)

    # Step 4: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""    
    cur.executemany(sql_string, rows_to_upload)
  
    # Step 5: Commit the changes and close the connection
    con.commit()
    con.close()



def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe
    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.
    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

In [98]:
# import data
address=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/address.csv')
customers=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/customers.csv')
ingredient=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/ingredient.csv')
inventory=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/inventory.csv')
item=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/item.csv')
orders=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/orders.csv')
recipe=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/recipe.csv')
rota=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/rota.csv')
shift=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/shift.csv')
staff=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/staff.csv')

In [99]:
# import data to sql tables
pd_to_sqlDB(address,"address","pizza_db")
pd_to_sqlDB(customers,"customers","pizza_db")
pd_to_sqlDB(ingredient,"ingredient","pizza_db")
pd_to_sqlDB(inventory,"inventory","pizza_db")
pd_to_sqlDB(item,"item","pizza_db")
pd_to_sqlDB(orders,"orders","pizza_db")
pd_to_sqlDB(recipe,"recipe","pizza_db")
pd_to_sqlDB(rota,"rota","pizza_db")
pd_to_sqlDB(shift,"shift","pizza_db")
pd_to_sqlDB(staff,"staff","pizza_db")

#Query for 1st dashboard

In [100]:
# Query table for the 1st dashboard
query="""
SELECT
  o.order_id,
  o.quantity*i.item_price AS order_sales,
  o.quantity,
  i.item_cat,
  i.item_name,
  o.created_at,
  a.delivery_address1,
  -- a.delivery_address2, --no data in address 2
  a.delivery_city,
  a.delivery_zipcode,
  o.delivery
FROM orders o
LEFT JOIN item i ON o.item_id=i.item_id
LEFT JOIN address a ON o.add_id=a.add_id;
"""
result1=sql_query_to_pd(query,"pizza_db")

In [101]:
# Convert to appropriate data type
result1['created_at']=pd.to_datetime(result1['created_at'],format="%d/%m/%Y %H:%M")
result1['delivery_zipcode']=result1['delivery_zipcode'].astype('str')

In [102]:
# Table for 1st dashboard 
print(result1)

       order_id  order_sales  quantity item_cat               item_name  \
0           109           24         2    Pizza    Pizza Margherita Reg   
1           109           24         2    Pizza    Pizza Margherita Reg   
2           109           24         2    Pizza    Pizza Margherita Reg   
3           109           24         2    Pizza    Pizza Margherita Reg   
4           109           24         2    Pizza    Pizza Margherita Reg   
...         ...          ...       ...      ...                     ...   
20091       166            6         1    Drink  Coca Cola Regular 1.5l   
20092       166            6         1    Drink  Coca Cola Regular 1.5l   
20093       166            6         1    Drink  Coca Cola Regular 1.5l   
20094       166            6         1    Drink  Coca Cola Regular 1.5l   
20095       166            6         1    Drink  Coca Cola Regular 1.5l   

               created_at    delivery_address1 delivery_city delivery_zipcode  \
0     2022-08-10 1

##Dashboard 1

This dashboard contains the basic information about the pizza shop. It tells whether the pizza shop is earning or losing money.
1.   Total number of order
2.   Total sales
3.   Total item sold
4.   Average order value
5.   Sales by category
6.   Top selling item
7.   Orders by hour
8.   Sales by hour
9.   Orders by address
10.  Orders by delivery/pick up

##1. Total orders

In [103]:
ttl_no_order=len(result1['order_id'].unique())
print(f'The total number of order is {ttl_no_order}')

The total number of order is 58


##2. Total sales

In [104]:
ttl_sales=sum(result1['order_sales'])
print(f'The total sales is ${ttl_sales}')

The total sales is $381888


##3. Total item sold

In [105]:
ttl_item_sold=sum(result1['quantity'])
print(f'The total item sold is {ttl_item_sold}')

The total item sold is 40064


##4. Average order value

In [106]:
avg_order_val=result1.groupby(['order_id'])['order_sales'].sum().mean()
print(f'The average order value is ${avg_order_val:.2f}')

The average order value is $6584.28


##5. Sales by category

In [107]:
sales_by_cat=result1.groupby(['item_cat'])['order_sales'].sum().to_frame().sort_values('order_sales',ascending=False)
print(f'The average order value is \n',sales_by_cat)

The average order value is 
           order_sales
item_cat             
Pizza          244224
Dessert         58560
Side            51264
Drink           27840


##6. Top selling item

In [108]:
top_item=result1.groupby(['item_name'])['order_sales'].sum().idxmax()
print(f'The top selling item is {top_item}')

The top selling item is Pizza Quattro Formaggi Large


##7. Order by hour

In [109]:
order_by_hour=result1.groupby(result1['created_at'].dt.hour)['order_id'].count().to_frame('count').rename_axis('hour')
print(f'The order by hour is\n',order_by_hour)

The order by hour is
       count
hour       
12     1920
13     6144
14      704
18      960
19     4288
20     2368
21     1792
22     1920


##8. Sales by hour

In [110]:
sales_by_hour=result1.groupby(result1['created_at'].dt.hour)['order_sales'].sum().to_frame('sales').rename_axis('hour')
print(f'The sales by hour is\n',sales_by_hour)

The sales by hour is
       sales
hour       
12    31168
13    89536
14     5824
18    26816
19    99328
20    36288
21    30080
22    62848


##9. Order by address

In [111]:
order_by_address=result1['delivery_address1'].value_counts().to_frame('count').rename_axis('address')
print(f'The order by address is\n',order_by_address)

The order by address is
                             count
address                          
150 Carter Street            1088
68 Princeton Street           768
18 Cambridge Street           768
125 Summer Street             704
425 Middle Turnpike East      576
95 Briarwood Drive            576
60 Desousa Drive              576
61 Plymouth Lane              576
697 Parker Street             576
29 Lucian Street              512
89 High Ledge Circle          448
44 Downey Drive               448
123 Elizabeth Drive           448
211 Oak Street                448
145 Saint John Street         448
86 Highland Street            384
22 Star Farms Drive           384
61 Hills Street               384
225 Kennedy Road              384
184 Woodland Street           384
126 Garth Road                384
65 Arcellia Drive             384
34 Holyoke Road               320
4 Orchard Street              320
25 Edwards Street             320
117 Adelaide Road             320
310 Timrod Road        

##10. Order by delivery/pick up

In [112]:
order_by_delivery=result1['delivery'].value_counts().to_frame('count').rename_axis('delivery(1)/pick up(0)')
print(f'The order by delivery/pick up is\n',order_by_delivery)

The order by delivery/pick up is
                         count
delivery(1)/pick up(0)       
1                       15296
0                        4800


#Query for 2nd dashboard

In [113]:
# Query table for the 2nd dashboard
query="""
SELECT
  sub1.item_name,
  sub1.ing_id,
  sub1.ing_name,
  sub1.ing_weight,
  sub1.ing_price,
  sub1.order_quan,
  sub1.rep_quan,
  sub1.order_quan*sub1.rep_quan AS quantity_required,
  sub1.ing_price/sub1.ing_weight AS unit_cost,
  (sub1.order_quan*sub1.rep_quan)*(sub1.ing_price/sub1.ing_weight) AS ingredient_cost
FROM
    (SELECT
      o.item_id,
      rep.recipe_id,
      i.item_name,
      rep.ing_id,
      ing.ing_name,
      SUM(o.quantity)  AS order_quan,
      rep.quantity AS rep_quan,
      ing.ing_price,
      ing.ing_weight
    FROM orders o
    LEFT JOIN item i ON o.item_id=i.item_id
    LEFT JOIN recipe rep ON i.sku=rep.recipe_id
    LEFT JOIN ingredient ing ON rep.ing_id=ing.ing_id
    GROUP BY
      o.item_id, 
      rep.recipe_id, 
      i.item_name,
      rep.ing_id,
      rep.quantity,
      ing.ing_name,
      ing.ing_weight,
      ing.ing_price) sub1;
"""
result2=sql_query_to_pd(query,"pizza_db")
result2['item_cost']=result2['rep_quan']*result2['unit_cost']
print(result2)

                  item_name  ing_id                   ing_name  ing_weight  \
0      Pizza Margherita Reg  ING001  Pizza dough ball (8 pack)        2000   
1      Pizza Margherita Reg  ING002              Tomato sauce         4500   
2      Pizza Margherita Reg  ING003          Mozzarella cheese        2500   
3      Pizza Margherita Reg  ING004              Dried oregano         500   
4    Pizza Margherita Large  ING001  Pizza dough ball (8 pack)        2000   
..                      ...     ...                        ...         ...   
104         Fanta Diet 1.5l  ING043            Fanta Diet 1.5l           1   
105     San Pelligrino 33cl  ING044        San Pelligrino 33cl           1   
106     San Pelligrino 1.5l  ING045        San Pelligrino 1.5l           1   
107            Perrier 33cl  ING046               Perrier 33cl           1   
108            Perrier 1.5l  ING047               Perrier 1.5l           1   

     ing_price  order_quan  rep_quan  quantity_required  unit_c

##Dashboard 2

The second dashboard is about the material cost and stock information to keep the pizza shop running.
1.   Total quantity required for orders by ingredient
2.   Total cost of ingredients for orders
3.   Calculated cost of pizza
4.   Percentage stock remaining by ingredient
5.   List of ingredients to re-order based on remaining inventory

##1. Total quantity required for orders by ingredient

In [114]:
quantity_by_ingredient=result2.groupby('ing_name')['quantity_required'].sum().to_frame()
print(f'The total quantity required for orders by ingredient is\n',quantity_by_ingredient)

The total quantity required for orders by ingredient is
                            quantity_required
ing_name                                    
Anchovies                             473600
Banoffee pie                         1075200
Caesar dressing                       189440
Calamari                              320000
Capers                                 34048
Chicken wings                         921600
Chilli pepper                         180480
Chocolate brownie                     825600
Chocolate ice cream                   947200
Coca Cola Diet 1.5l                      256
Coca Cola Diet 33cl                     2048
Coca Cola Regular 1.5l                  2560
Coca Cola Regular 33cl                  3584
Croutons                              473600
Dried oregano                          37376
Eggplant                              660480
Fanta Diet 1.5l                          512
Fanta Regular 1.5l                      1536
Fanta Regular 33cl                      742

##2. Total cost of ingredients for orders

In [115]:
cost_by_ingredient=result2.groupby('ing_name')['ingredient_cost'].sum().to_frame()
print(f'The total cost of ingredients for orders is\n',cost_by_ingredient.round(2))

The total cost of ingredients for orders is
                            ingredient_cost
ing_name                                  
Anchovies                          5204.86
Banoffee pie                       1030.40
Caesar dressing                     896.35
Calamari                           3682.56
Capers                              141.64
Chicken wings                     10725.89
Chilli pepper                      1171.32
Chocolate brownie                  1469.57
Chocolate ice cream                3252.05
Coca Cola Diet 1.5l                 245.76
Coca Cola Diet 33cl                 839.68
Coca Cola Regular 1.5l             2457.60
Coca Cola Regular 33cl             1469.44
Croutons                           1989.12
Dried oregano                       447.76
Eggplant                           1254.91
Fanta Diet 1.5l                     491.52
Fanta Regular 1.5l                 1474.56
Fanta Regular 33cl                 3043.84
Fruit salad                        1038.95
Garlic an

##3. Calculated cost of pizza

In [116]:
cost_of_pizza=result2.groupby('item_name')[['item_cost']].sum()
filt=(cost_of_pizza.index.str.contains("Pizza"))
print(f'The cost_of_pizza is\n',cost_of_pizza[filt].round(2))

The cost_of_pizza is
                               item_cost
item_name                              
Pizza Diavola (hot) Large          2.73
Pizza Diavola (hot) Reg            2.18
Pizza Hawaiian Large               3.00
Pizza Hawaiian Reg                 2.55
Pizza Margherita Large             1.97
Pizza Margherita Reg               1.64
Pizza Napolitana Large             2.45
Pizza Napolitana Reg               2.70
Pizza Parmigiana Large             3.66
Pizza Parmigiana Reg               3.08
Pizza Pepperoni Large              4.20
Pizza Pepperoni Reg                3.51
Pizza Quattro Formaggi Large       5.13
Pizza Quattro Formaggi Reg         4.29
Pizza Seafood Large                6.13
Pizza Seafood Reg                  5.23


In [117]:
# use previous query for following questions
pd_to_sqlDB(result2,"result2","pizza_db")

# query to answer remaining question in dashboard2
query="""
SELECT
  sub2.ing_name,
  sub2.ordered_weight,
  (ing.ing_weight*inv.quantity) AS total_inv_weight
FROM
  (SELECT
    ing_id,
    ing_name,
    SUM(quantity_required) AS ordered_weight
  FROM result2
  GROUP BY 
  ing_name, ing_id) sub2
LEFT JOIN inventory inv ON inv.item_id = sub2.ing_id
LEFT JOIN ingredient ing ON ing.ing_id = sub2.ing_id;
"""
result3=sql_query_to_pd(query,"pizza_db")
print(result3)

              ing_name  ordered_weight  total_inv_weight
0            Anchovies          654900              2000
1            Anchovies          654900              2000
2            Anchovies          654900              2000
3            Anchovies          654900              2000
4            Anchovies          654900              2000
..                 ...             ...               ...
715  Vanilla ice cream          283200              9000
716  Vanilla ice cream          283200              9000
717  Vanilla ice cream          283200              9000
718  Vanilla ice cream          283200              9000
719  Vanilla ice cream          283200              9000

[720 rows x 3 columns]


##4. Percentage stock remaining by ingredient

In [118]:
result3['perc_remain']=((result3['total_inv_weight']-result3['ordered_weight'])/result3['total_inv_weight']*100)
print(f'The percentage remaining is\n',result3[['ing_name','perc_remain']].round(2))

The percentage remaining is
               ing_name  perc_remain
0            Anchovies    -32645.00
1            Anchovies    -32645.00
2            Anchovies    -32645.00
3            Anchovies    -32645.00
4            Anchovies    -32645.00
..                 ...          ...
715  Vanilla ice cream     -3046.67
716  Vanilla ice cream     -3046.67
717  Vanilla ice cream     -3046.67
718  Vanilla ice cream     -3046.67
719  Vanilla ice cream     -3046.67

[720 rows x 2 columns]


##5. List of ingredients to re-order based on remaining inventory

In [119]:
print(f'The ingredients need to re-order is\n',result3[['ing_name','perc_remain']].sort_values("perc_remain").round(2))

The ingredients need to re-order is
                 ing_name  perc_remain
23          Banoffee pie     -61850.0
31          Banoffee pie     -61850.0
30          Banoffee pie     -61850.0
29          Banoffee pie     -61850.0
28          Banoffee pie     -61850.0
..                   ...          ...
157  Coca Cola Diet 1.5l       -195.0
158  Coca Cola Diet 1.5l       -195.0
159  Coca Cola Diet 1.5l       -195.0
146  Coca Cola Diet 1.5l       -195.0
609     Sprite Diet 33cl       -195.0

[720 rows x 2 columns]


#Query for 3rd dashboard

In [120]:
# Query table for the 3rd dashboard
query="""
SELECT
  st.first_name,
  st.last_name,
  st.hourly_rate,
  sh.day_of_week,
  sh.start_time,
  sh.end_time
FROM staff st
LEFT JOIN rota ro ON ro.staff_id=st.staff_id
LEFT JOIN shift sh ON sh.shift_id=ro.shift_id
"""
result4=sql_query_to_pd(query,"pizza_db")
result4['start_time']=pd.to_timedelta(result4['start_time'].str.strip())
result4['end_time']=pd.to_timedelta(result4['end_time'].str.strip())
result4['hour_worked']=(result4['end_time']-result4['start_time'])/np.timedelta64(1, 'h')
print(result4)

     first_name last_name  hourly_rate day_of_week      start_time  \
0         Mindy     Sloan        17.25   Wednesday 0 days 10:30:00   
1         Mindy     Sloan        17.25   Wednesday 0 days 10:30:00   
2         Mindy     Sloan        17.25   Wednesday 0 days 10:30:00   
3         Mindy     Sloan        17.25   Wednesday 0 days 10:30:00   
4         Mindy     Sloan        17.25   Wednesday 0 days 10:30:00   
...         ...       ...          ...         ...             ...   
2587  Johnathon  Bradford        14.50      Sunday 0 days 18:30:00   
2588    Matilda   Mccarty        14.50        None             NaT   
2589      Areeb   Vasquez        14.50        None             NaT   
2590     Amiyah   Lambert        14.50        None             NaT   
2591      Amrit   Greaves        14.50        None             NaT   

            end_time  hour_worked  
0    0 days 14:30:00          4.0  
1    0 days 14:30:00          4.0  
2    0 days 14:30:00          4.0  
3    0 days 14:

##Dashboard 3

The last dashboard is going to capture the human resource cost of the pizza shop.
1.   Hours worked by staff member
2.   Total hours worked
3.   Cost per staff member
4.   Total staff cost

In [121]:
hours_worked_by_staff=result4.groupby(['first_name'])['hour_worked'].sum()
print(f'The hours worked by staff is\n',hours_worked_by_staff)

The hours worked by staff is
 first_name
Amiyah           0.0
Amrit            0.0
Areeb            0.0
Arran         1088.0
Desiree       1632.0
Faraz            0.0
Ivan          1088.0
Johnathon     1088.0
Lilly-Rose    1632.0
Luqman        1632.0
Matilda          0.0
Mindy         1632.0
Sana             0.0
Seren         1088.0
Talha            0.0
Zachery          0.0
Name: hour_worked, dtype: float64


##2. Total hours worked

In [122]:
ttl_hour_worked=hours_worked_by_staff.sum()
print(f'The total hour worked is {ttl_hour_worked}')

The total hour worked is 10880.0


##3. Cost per staff member

In [123]:
cost_tab=result4.groupby(['first_name'])[['hour_worked','hourly_rate']].agg({'hour_worked':'sum','hourly_rate':'mean'})
cost_tab['cost']=cost_tab['hour_worked'].multiply(cost_tab['hourly_rate']).round(2)
print(f'The cost per staff is \n',cost_tab)

The cost per staff is 
             hour_worked  hourly_rate     cost
first_name                                   
Amiyah              0.0        14.50      0.0
Amrit               0.0        14.50      0.0
Areeb               0.0        14.50      0.0
Arran            1088.0        21.50  23392.0
Desiree          1632.0        14.50  23664.0
Faraz               0.0        21.50      0.0
Ivan             1088.0        14.50  15776.0
Johnathon        1088.0        14.50  15776.0
Lilly-Rose       1632.0        14.50  23664.0
Luqman           1632.0        21.50  35088.0
Matilda             0.0        14.50      0.0
Mindy            1632.0        17.25  28152.0
Sana                0.0        21.50      0.0
Seren            1088.0        17.25  18768.0
Talha               0.0        17.25      0.0
Zachery             0.0        17.25      0.0


##4. Total staff cost

In [124]:
ttl_staff_cost=cost_tab['cost'].sum()
print(f'The total staff cost is ${ttl_staff_cost}')

The total staff cost is $184280.0


#Export csv for visualization

In [125]:
result1.to_csv('/content/gdrive/My Drive/SQL/export/pizza1.csv',index=False)
result2.to_csv('/content/gdrive/My Drive/SQL/export/pizza2.csv',index=False)
result3.to_csv('/content/gdrive/My Drive/SQL/export/pizza3.csv',index=False)
result4.to_csv('/content/gdrive/My Drive/SQL/export/pizza4.csv',index=False)

link for visualization<br>
https://public.tableau.com/views/Book1_16823748427290/PizzaShopBI?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

#Data source

https://learnbi.online/pizzaproject